In [1]:
import os
import pandas as pd
import re

df_articles = pd.read_pickle("../data/processed/parsed_articles.pkl")

# 1. Fixing text data

Also remove training spaces

Apply to all text fields

Additional standardized title:
    * Remove title stop words
    * Remove load data at the end

In [2]:
from ftfy import fix_text
df_articles['body'] = df_articles['body'].apply(fix_text)
df_articles['title'] = df_articles['title'].apply(fix_text)

In [3]:
# standardize titles for near-duplicates identification

def standardize_text(text):
    # Remove all non-alphanumeric characters except spaces, then lowercase
    return re.sub(r'[^a-zA-Z0-9 ]', '', text).lower()

df_articles['title_stand'] = df_articles['title'].apply(standardize_text)

# 2. Additiona Cleaning

* Dates


* Publishers --> group



# 2. Filters


* number of words

* title is

* title contains



# 3. Deduplication

* Standard

* Near duplicate

# 4. Seperate dataset